In [341]:
import numpy as np
import pandas as pd
import re
from greedypacker import *
from greedypacker import greedypacker

import warnings
warnings.filterwarnings('ignore')

In [342]:
#загружаем из эксель листы с данными
data = 'data/data_source.xlsx'

hangars =pd.read_excel(data, sheet_name='Ангары', index_col=None)
aircompany = pd.read_excel(data, sheet_name='Авиакомпании', index_col=None)
aircrafts = pd.read_excel(data, sheet_name='Типы ВС', index_col=None)
maintenance =  pd.read_excel(data, sheet_name='Типы ТО', index_col=None)
needs = pd.read_excel(data, sheet_name='Потребность Авиакомпаний', index_col=None)

In [343]:
#заполняем пропуски в данных 0
maintenance.fillna(0, inplace = True)
needs.fillna(0,inplace=True)
needs = needs.iloc[:-1, :-1]
maintenance = maintenance.iloc[:, :-1]

In [344]:
needs['serviced'] = 0

In [345]:
new_df = pd.merge(needs, maintenance, how='inner', on=['Формат ТО','Тип ВС'])

In [346]:
new_df = pd.merge(new_df, aircompany, how='inner')

In [347]:
new_df = pd.merge(new_df, aircrafts, how='inner')

In [348]:
new_df['aircraft square'] = new_df['Длина, м']*new_df['Размах крыла, м']

In [349]:
new_df['SVO revenue']= new_df['Длительность, \nдней']*new_df['SVO \n(стоимость 1 дня), руб']
new_df['DME revenue']= new_df['Длительность, \nдней']*new_df['DME \n(стоимость 1 дня), руб']
new_df['VKO revenue']= new_df['Длительность, \nдней']*new_df['VKO \n(стоимость 1 дня), руб']

In [350]:
new_df['season duration'] = new_df['Окончание сезона технического обслуживания'] - new_df['Начало сезона технического обслуживания']

In [351]:
cols = new_df.columns

In [352]:
df = pd.DataFrame(np.repeat(new_df.values, new_df['Потребность авиакомпании, штук'].values, axis=0), columns=cols)

In [353]:
df['Total revenue']= df['SVO revenue']+df['DME revenue']+df['VKO revenue']

In [354]:
cols = ['Формат ТО', 'Тип ВС','SVO \n(стоимость 1 дня), руб', 'DME \n(стоимость 1 дня), руб', 'VKO \n(стоимость 1 дня), руб',
        'Авиакомпания','Потребность авиакомпании, штук',
        'Минимальное количество форм ТО, \nкоторое необходимо выполнить по контракту, штук',
               'serviced',
        'Длина, м', 'Размах крыла, м', 'aircraft square',
       'Длительность, \nдней',
       'Начало сезона технического обслуживания',
       'Окончание сезона технического обслуживания', 'season duration', 
        'Штрафной коэффициент',
        'SVO revenue',
       'DME revenue', 'VKO revenue','Total revenue']

In [355]:
df = df[cols]

In [356]:
df.columns = ['ТО', 'ВС', 'стоимость_SVO',
       'стоимость_DME', 'стоимость_VKO',
       'Авиакомпания', 'Потребность авиакомпании, штук',
       'minimum plan',
       'serviced', 'dlina', 'razmah', 'aircraft_square',
       'duration', 'start_season',
       'end_season', 'season duration',
       'penalty', 'SVO_revenue', 'DME_revenue', 'VKO_revenue',
       'total_revenue']

In [357]:
df = df[['ТО', 'ВС', 'стоимость_SVO',
       'стоимость_DME', 'стоимость_VKO',
       'Авиакомпания', 'Потребность авиакомпании, штук',
       'minimum plan', 'serviced',
        'dlina', 'razmah', 'aircraft_square',
       'duration', 'start_season',
       'end_season', 'season duration',
       'penalty', 'SVO_revenue', 'DME_revenue', 'VKO_revenue',
       'total_revenue']]

In [358]:
df['min_plan']=0

In [359]:
df_test = df.copy()

In [360]:
key = df_test.iloc[0,0] + df_test.iloc[0,1] 
counter = df_test.iloc[0,7]

for i in range(0,df_test.shape[0],1):
    k = df_test.iloc[i,0] + df_test.iloc[i,1]
    if k == key:
        if counter !=0:
            df_test.iloc[i,21] = 1
            counter = counter - 1
        else:
            df_test.iloc[i,21] = 0
    else:
        key = k
        counter = df_test.iloc[i,7]
        if counter !=0:
            df_test.iloc[i,21] = 1
            counter = counter - 1
        else:
            df_test.iloc[i,21] = 0

In [361]:
df_test = df_test.drop(['Потребность авиакомпании, штук','minimum plan'], axis=1)

In [362]:
df_test = df_test[['ТО', 'ВС', 'стоимость_SVO', 'стоимость_DME', 'стоимость_VKO',
       'Авиакомпания','min_plan', 'serviced', 'dlina', 'razmah', 'aircraft_square',
       'duration', 'start_season', 'end_season', 'season duration', 'penalty',
       'SVO_revenue', 'DME_revenue', 'VKO_revenue', 'total_revenue']]

In [363]:
SVO_width = 200
SVO_deep = 90
SVO_area = SVO_width * SVO_deep

DME_width = 300
DME_deep = 80
DME_area = DME_width * DME_deep


df['SVO_rating'] = None
df['DME_rating'] = None
df['VKO_rating'] = None

for i in range(len(df)):
    df['SVO_rating'][i] = df.SVO_revenue[i] / (df.aircraft_square[i] + df.duration[i]) 
    df['DME_rating'][i] = df.DME_revenue[i] / (df.aircraft_square[i] + df.duration[i])
    df['VKO_rating'][i] = df.VKO_revenue[i] / (df.aircraft_square[i] + df.duration[i])  

df['days_since_start'] = 0

# flag of priority hangar for a plane
df['SVO'] = 0
df['DME'] = 0
df['VKO'] = 0
df['in_service'] = 0 # флаг, самолет в обслуживании в ангаре или нет (0 - нет, 1 - обслуживается)
df['x'] = -1 # координаты размещения самолета в ангаре
df['y'] = -1 # # координаты размещения самолета в ангаре


for i in range(len(df)):
    
    options = [df['SVO_rating'][i], df['DME_rating'][i], df['VKO_rating'][i]]
    best_option = options.index(max(options))
    
    if best_option == 0:
        df['SVO'][i] = 1
    elif best_option == 1:
        df['DME'][i] = 1
    else:
        df['VKO'][i] = 1

In [364]:
def SVO_packing(df): 

    SVO_hangar = df.query('SVO == 1 and in_service == 0 and serviced == 0')
    SVO_hangar.sort_values(by='SVO_rating', ascending = False, inplace = True) 

    plane_to_SVO = SVO_hangar.index #индексы DF самолетов - претендентов на ангар SVO

    SVO_plan = greedypacker.BinManager(SVO_deep, SVO_width, pack_algo='guillotine', heuristic='best_longside', rectangle_merge=True, rotation=True)

    levels = 1 # количество уровней размещения самолетов в ангаре (выбран 1)

    plane = 0 #счетчик самолетов по списку претендентов
    
    final_list = []
    
    while len(SVO_plan.bins) <= levels: 
        try:
            if SVO_plan.bins[1] != True:
                final_list = final_list[:-1]
                for item in final_list:
                    plane_width = SVO_hangar['razmah'][plane_to_SVO[item]]
                    plane_length = SVO_hangar['dlina'][plane_to_SVO[item]]
                    SVO_plan.add_items(greedypacker.Item(plane_width, plane_length))
                    SVO_plan.execute()    
                break
                
        except:
            plane_width = SVO_hangar['razmah'][plane_to_SVO[plane]]
            plane_length = SVO_hangar['dlina'][plane_to_SVO[plane]]

            SVO_plan.add_items(greedypacker.Item(plane_width, plane_length))
            SVO_plan.execute()
            final_list.append(plane_to_SVO[plane])
            plane += 1

    items = str(SVO_plan.bins[0]).split('Item')[1:]
    template = r'[a-zA-Z,=,(,)]'

    for item in items: 
        position = re.sub(template,'',item)
        position = re.sub(']','',position).split(' ')

        for plane in plane_to_SVO:

            razmah = round(df.loc[plane]['razmah'],2)
            dlina = round(df.loc[plane]['dlina'],2)

            if all([razmah == float(position[0]), dlina == float(position[1]), float(df.loc[plane]['in_service']) == 0]):
                df.loc[plane,'x'] = float(position[2])
                df.loc[plane,'y'] = float(position[3])
                df.loc[plane,'in_service'] = 1
                break
            else:
                continue
    print(SVO_plan.bins[1])
    return df

In [365]:
SVO_packing(df)

Guillotine([Item(width=34.1, height=37.57, x=0, y=0), Item(width=34.1, height=37.57, x=34.1, y=0)])


,ТО,ВС,стоимость_SVO,стоимость_DME,стоимость_VKO,Авиакомпания,"Потребность авиакомпании, штук",minimum plan,serviced,dlina,...,SVO_rating,DME_rating,VKO_rating,days_since_start,SVO,DME,VKO,in_service,x,y
0,C-Check,A319,30000,30000,28000,C7,1,1,0,33.84,...,586.264,586.264,547.18,0,1,0,0,0,-1.0,-1.0
1,D-Check,A319,40000,0,39000,C7,2,2,0,33.84,...,1340.1,0,1306.59,0,1,0,0,0,-1.0,-1.0
2,D-Check,A319,40000,0,39000,C7,2,2,0,33.84,...,1340.1,0,1306.59,0,1,0,0,0,-1.0,-1.0
3,C-Check,A319,30000,30000,28000,Уральские Авиалинии,2,1,0,33.84,...,460.773,460.773,430.055,0,1,0,0,0,-1.0,-1.0
4,C-Check,A319,30000,30000,28000,Уральские Авиалинии,2,1,0,33.84,...,460.773,460.773,430.055,0,1,0,0,0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Painting,737-300,19000,18000,15000,Победа,3,0,0,33.25,...,156.983,148.72,123.934,0,1,0,0,0,-1.0,-1.0
236,C-Check,737-300,0,32000,32000,Победа,4,2,0,33.25,...,0,969.442,969.442,0,0,1,0,0,-1.0,-1.0
237,C-Check,737-300,0,32000,32000,Победа,4,2,0,33.25,...,0,969.442,969.442,0,0,1,0,0,-1.0,-1.0
238,C-Check,737-300,0,32000,32000,Победа,4,2,0,33.25,...,0,969.442,969.442,0,0,1,0,0,-1.0,-1.0


In [366]:
def DME_packing(df): 

    DME_hangar = df.query('DME == 1 and in_service == 0 and serviced == 0')
    DME_hangar.sort_values(by='DME_rating', ascending = False, inplace = True) 

    plane_to_DME = DME_hangar.index #индексы DF самолетов - претендентов на ангар DME

    DME_plan = greedypacker.BinManager(DME_deep, DME_width, pack_algo='guillotine', heuristic='best_longside', rectangle_merge=True, rotation=True)

    levels = 1 # количество уровней размещения самолетов в ангаре (выбран 1)

    plane = 0 #счетчик самолетов по списку претендентов
    
    while len(DME_plan.bins) <= levels: #пока не заполниться первый ряд

            plane_width = DME_hangar['razmah'][plane_to_DME[plane]]
            plane_length = DME_hangar['dlina'][plane_to_DME[plane]]

            DME_plan.add_items(greedypacker.Item(plane_width, plane_length))
            DME_plan.execute()
            plane += 1

    items = str(DME_plan.bins[0]).split('Item')[1:]
    template = r'[a-zA-Z,=,(,)]'

    for item in items: 
        position = re.sub(template,'',item)
        position = re.sub(']','',position).split(' ')

        for plane in plane_to_DME:

            razmah = round(df.loc[plane]['razmah'],2)
            dlina = round(df.loc[plane]['dlina'],2)

            if all([razmah == float(position[0]), dlina == float(position[1]), float(df.loc[plane]['in_service']) == 0]):
                df.loc[plane,'x'] = float(position[2])
                df.loc[plane,'y'] = float(position[3])
                df.loc[plane,'in_service'] = 1
                break
            else:
                continue      
    return df

In [367]:
df = SVO_packing(df)
#df = SVO_packing(df)

Guillotine([Item(width=34.1, height=37.57, x=0, y=0), Item(width=34.1, height=37.57, x=34.1, y=0), Item(width=34.1, height=37.57, x=0, y=37.57), Item(width=34.1, height=33.84, x=34.1, y=37.57), Item(width=34.1, height=33.84, x=0, y=75.14)])


In [368]:
def check_out(df):
    check_out = df.query('in_service == 1').duration.min()
    df[(df.duration == check_out) & (df.in_service == 1)]['serviced'] = 1

In [225]:
DME_hangar = df.query('DME == 1 and in_service == 0 and serviced == 0')
DME_hangar.sort_values(by='DME_rating', ascending = False, inplace = True) 

plane_to_DME = DME_hangar.index #индексы DF самолетов - претендентов на ангар DME

DME_plan = greedypacker.BinManager(DME_deep, DME_width, pack_algo='guillotine', heuristic='best_longside', rectangle_merge=True, rotation=True)

levels = 1 # количество уровней размещения самолетов в ангаре (выбран 1 этаж)

plane = 0 #счетчик самолетов по списку претендентов на ремонт
    
temp_list = []
while len(DME_plan.bins) <= levels: #пока не заполниться первый ряд
    
    plane_width = DME_hangar['razmah'][plane_to_DME[plane]]
    plane_length = DME_hangar['dlina'][plane_to_DME[plane]]
    
    DME_plan.add_items(greedypacker.Item(plane_width, plane_length))
    DME_plan.execute()
    plane += 1

items = str(DME_plan.bins[0]).split('Item')[1:]
template = r'[a-zA-Z,=,(,)]'

for item in items: 
    position = re.sub(template,'',item)
    position = re.sub(']','',position).split(' ')

    for plane in plane_to_DME:

        razmah = round(df.loc[plane]['razmah'],2)
        dlina = round(df.loc[plane]['dlina'],2)

        if all([razmah == float(position[0]), dlina == float(position[1]), float(df.loc[plane]['in_service']) == 0]):
            df.loc[plane,'x'] = float(position[2])
            df.loc[plane,'y'] = float(position[3])
            df.loc[plane,'in_service'] = 1
            break
        else:
            continue 

In [370]:
VKO_width = 150
VKO_deep = 70
VKO_area = DME_width * DME_deep

VKO_hangar = df.query('VKO == 1 and in_service == 0 and serviced == 0')
VKO_hangar.sort_values(by='VKO_rating', ascending = False, inplace = True) 

plane_to_VKO = VKO_hangar.index #индексы DF самолетов - претендентов на ангар DME

VKO_plan = greedypacker.BinManager(VKO_deep, VKO_width, pack_algo='guillotine', heuristic='best_longside', rectangle_merge=True, rotation=True)

levels = 1 # количество уровней размещения самолетов в ангаре (выбран 1 этаж)

plane = 0 #счетчик самолетов по списку претендентов на ремонт
    
temp_list = []
while len(VKO_plan.bins) <= levels: #пока не заполниться первый ряд
    
    plane_width = VKO_hangar['razmah'][plane_to_VKO[plane]]
    plane_length = VKO_hangar['dlina'][plane_to_VKO[plane]]
    
    VKO_plan.add_items(greedypacker.Item(plane_width, plane_length))
    VKO_plan.execute()
    plane += 1

items = str(VKO_plan.bins[0]).split('Item')[1:]
template = r'[a-zA-Z,=,(,)]'

for item in items: 
    position = re.sub(template,'',item)
    position = re.sub(']','',position).split(' ')

    for plane in plane_to_VKO:

        razmah = round(df.loc[plane]['razmah'],2)
        dlina = round(df.loc[plane]['dlina'],2)

        if all([razmah == float(position[0]), dlina == float(position[1]), float(df.loc[plane]['in_service']) == 0]):
            df.loc[plane,'x'] = float(position[2])
            df.loc[plane,'y'] = float(position[3])
            df.loc[plane,'in_service'] = 1
            break
        else:
            continue 

In [381]:
VKO_plan.bins[0] = VKO_plan.execute()

In [383]:
VKO_plan.bins[0]